Grab most recent data

Add most recent data to competes

In [10]:
logging.info(f"Addding latest race{race_data['RaceName']} {race_data['Date'].split('-')[0]} data to table")
try:
    c.execute("""
          INSERT INTO Competes_Record
          (driverID, raceID, position, points)
          SELECT Driver.driverID, %s, temp_positions.place, temp_positions.points
          FROM temp_positions
          INNER JOIN Driver
            ON Driver.name=temp_positions.name
          """, (raceId,))
except sqlalchemy.exc.IntegrityError:
    logging.log(logging.INFO, "Repeated keys, double check new race is updated")
except Exception as e:
    logging.log(logging.WARN, f"Internal DB Error: {e}")


Grab all updated driver bets cases

In [24]:
logging.log(logging.INFO, f"Addding latest race{race_data['RaceName']} {race_data['Date'].split('-')[0]} data to table")
df1=None
try: # CLOSE EXISTING BETS
    df1 = pd.read_sql(f"""
          SELECT Bet.betId, (CASE WHEN isOver THEN temp_positions.place<=Bet.place ELSE temp_positions.place>=Bet.place END) As isWon 
          FROM temp_positions
          INNER JOIN Driver
            ON Driver.name=temp_positions.name
          INNER JOIN Informs
            ON Informs.driverId=Driver.driverId
            AND Informs.raceId={raceId}
          INNER JOIN Bet
            ON Informs.BetId=Bet.BetId
          WHERE Bet.teamName IS NULL
          """, c)
except sqlalchemy.exc.IntegrityError:
    logging.log(logging.INFO, "Repeated keys, double check new race is updated")
except Exception as e:
    logging.log(logging.WARN, f"Internal DB Error: {e}")
print(df1)


   betid  iswon
0     88   True
1     87   True
2     84   True
3     83   True
4     82  False
5     44  False


grab all team bets cases

In [20]:
logging.log(logging.INFO, f"Addding latest race{race_data['RaceName']} {race_data['Date'].split('-')[0]} data to table")
df2=None
try: # CLOSE EXISTING BETS
    df2 = pd.read_sql("""
            SELECT Bet.betId As betId, (CASE WHEN isOver THEN T.rank<=Bet.place ELSE T.rank>=Bet.place END) As isWon 
            FROM (SELECT DrivesFor.teamName AS teamName, ROW_NUMBER() OVER(ORDER BY SUM(temp_positions.points) DESC) AS rank
                 FROM temp_positions
                 INNER JOIN Driver
                     ON Driver.name=temp_positions.name
                 INNER JOIN DrivesFor
                     ON Driver.driverId=DrivesFor.driverId
                 GROUP BY DrivesFor.teamName) T
            INNER JOIN Informs
                ON T.teamName=Informs.teamName
            INNER JOIN Bet
                ON Informs.BetId=Bet.BetId
            WHERE Bet.driverId IS NULL
          """, c)
except sqlalchemy.exc.IntegrityError:
    logging.log(logging.INFO, "Repeated keys, double check new race is updated")
except Exception as e:
    logging.log(logging.WARN, f"Internal DB Error: {e}")


update all cases and close all open bets in informs

In [25]:
# UPDATE
df3=pd.concat([df1, df2])
df3.to_sql('temp_results', con=c, if_exists='replace')

c.execute("""
          UPDATE Bet
          SET isWon = temp_results.isWon,
              completed=True
          FROM 
            temp_results
          WHERE
            Bet.betId=temp_results.betId
          """)

Close all bets that might've been missed

In [26]:
c.execute(f"""
          UPDATE Bet
          SET completed=True
          WHERE Bet.raceId={raceId}
          """)
          


OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: 
          UPDATE Bet
          SET completed=True
          WHERE Bet.raceId=3
          ]
(Background on this error at: https://sqlalche.me/e/14/e3q8)